In [1]:
wikiurl = 'http://en.wikipedia.org/wiki/Kevin_Bacon'

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen(wikiurl)
bs = BeautifulSoup(html, "lxml")
for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

#bodyContent
/wiki/Main_Page
/wiki/Wikipedia:Contents
/wiki/Portal:Current_events
/wiki/Special:Random
/wiki/Wikipedia:About
//en.wikipedia.org/wiki/Wikipedia:Contact_us
/wiki/Help:Contents
/wiki/Help:Introduction
/wiki/Wikipedia:Community_portal
/wiki/Special:RecentChanges
/wiki/Wikipedia:File_upload_wizard
/wiki/Main_Page
/wiki/Special:Search
https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en
/w/index.php?title=Special:CreateAccount&returnto=Kevin+Bacon
/w/index.php?title=Special:UserLogin&returnto=Kevin+Bacon
https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en
/w/index.php?title=Special:CreateAccount&returnto=Kevin+Bacon
/w/index.php?title=Special:UserLogin&returnto=Kevin+Bacon
/wiki/Help:Introduction
/wiki/Special:MyContributions
/wiki/Special:MyTalk
#
#Early_life_and_education
#Acting_career
#Earl

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen(wikiurl)
bs = BeautifulSoup(html, "lxml")
for link in bs.find('div', {'id': 'bodyContent'}).find_all(
    'a', href=re.compile('^(/wiki/)((?!:).)*$')
):
    print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/Philadelphia
/wiki/Kevin_Bacon_filmography
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Leading_man
/wiki/Character_actor
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/National_Lampoon%27s_Animal_House
/wiki/Diner_(1982_film)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Frost/Nixon_(film)
/wiki/Friday_the_13th_(1980_film)
/wiki/Tremors_(1990_film)
/wiki/The_River_Wild
/wiki/The_Woodsman_(2004_film)
/wiki/Crazy,_Stupid,_Love
/wiki/Patriots_Day_(film)
/wiki/Losing_Chase
/wiki/Loverboy_(2005_film)
/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Miniseries_or_Television_Film
/wiki/Screen_Actors_Guild_Award_for_Outstanding_Performance_by_a_Male_Actor_in_a_Miniseries_or_Television_Movie
/wiki/Michael_Strobl
/wiki/HBO
/wiki/Taking_Chance
/wiki/Fox_Broadcasting_Company
/wik

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    html = urlopen(f'http://en.wikipedia.org{pageUrl}')
    bs = BeautifulSoup(html, "lxml")
    try:
        print(bs.h1.get_text())
        bodyContent = bs.find('div', {'id':'bodyContent'}).find_all('p')
        if len(bodyContent):
            print(bodyContent[0])
        print(bs.find(id='ca-edit').find('a').attrs['href'])
    except AttributeError:
        print('This page is missing something! Continuing')
        
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
            
getLinks('/wiki/General-purpose_programming_language')

General-purpose programming language
<p>In <a class="mw-redirect" href="/wiki/Computer_software" title="Computer software">computer software</a>, a <b>general-purpose programming language</b> (<b>GPL</b>) is a <a href="/wiki/Programming_language" title="Programming language">programming language</a> for building <a href="/wiki/Software" title="Software">software</a> in a wide variety of application <a href="/wiki/Domain_(software_engineering)" title="Domain (software engineering)">domains</a>. Conversely, a <a href="/wiki/Domain-specific_language" title="Domain-specific language">domain-specific programming language</a> (DSL) is used within a specific area. For example, <a href="/wiki/Python_(programming_language)" title="Python (programming language)">Python</a> is a GPL, while <a href="/wiki/SQL" title="SQL">SQL</a> is a DSL for <a href="/wiki/Query_language" title="Query language">querying relational databases</a>.
</p>
/w/index.php?title=General-purpose_programming_language&action=

HTTPError: HTTP Error 404: Not Found

In [6]:
pages

{'/wiki/Help:Contents',
 '/wiki/Help:Introduction',
 '/wiki/Main_Page',
 '/wiki/Portal:Current_events',
 '/wiki/Special:MyContributions',
 '/wiki/Special:MyTalk',
 '/wiki/Special:Random',
 '/wiki/Special:RecentChanges',
 '/wiki/Special:Search',
 '/wiki/Special:WhatLinksHere/User_talk:2400:2650:6100:B000:BF3A:186D:718A:D9A1',
 '/wiki/Wikipedia:About',
 '/wiki/Wikipedia:Community_portal',
 '/wiki/Wikipedia:Contents',
 '/wiki/Wikipedia:File_upload_wizard'}

In [18]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

def getInternalLinks(bs, url):
    netloc = urlparse(url).netloc
    schema = urlparse(url).scheme
    internalLinks = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc == '':
            internalLinks.add(f'{schema}://{netloc}/{link.attrs["href"].strip("/")}')
        elif parsed.netloc == netloc:
            internalLinks.add(link.attrs['href'])
    return list(internalLinks)

def getExternalLinks(bs, url):
    netloc = urlparse(url).netloc
    externalLinks = set()
    for link in bs.find_all('a'):
        if not link.attrs.get('href'):
            continue
        parsed = urlparse(link.attrs['href'])
        if parsed.netloc != '' and parsed.netloc != netloc:
            externalLinks.add(link.attrs['href'])
    return list(externalLinks)

def getRandomExternalLink(startingPage):
    bs = BeautifulSoup(urlopen(startingPage), "lxml")
    externalLinks = getExternalLinks(bs, startingPage)
    if not len(externalLinks):
        print('No external links, looking around the site for one')
        internalLinks = getInternalLinks(bs, startingPage)
        return getRandomExternalLink(random.choice(internalLinks))
    else:
        return random.choice(externalLinks)
    
def followExternalOnly(startingSite):
    externalLinks = getRandomExternalLink(startingSite)
    print(f'Random external link is: {externalLinks}')
    followExternalOnly(externalLinks)
    
followExternalOnly('https://www.oreilly.com/')

Random external link is: https://learning.oreilly.com/search/?query=author%3A%22Sari%20Greene%22&extended_publisher_data=true&highlight=true&include_assessments=false&include_case_studies=true&include_courses=true&include_playlists=true&include_collections=true&include_notebooks=true&include_sandboxes=true&include_scenarios=true&is_academic_institution_account=false&source=user&sort=date_added&facet_json=true&json_facets=true&page=0&include_facets=false
Random external link is: https://www.oreilly.com/modern-slavery-act-transparency-statement.html
Random external link is: https://oreilly.id/
Random external link is: https://www.oreilly.com/anz/
Random external link is: https://learning.oreilly.com/search/?q=author%3A%22Bruno%20Gon%C3%A7alves%22&type=*
Random external link is: https://twitter.com/oreillymedia
No external links, looking around the site for one


IndexError: Cannot choose from an empty sequence

In [10]:
allExtLinks = []
allIntLinks = []

def getAllExternalLinks(url):
    bs = BeautifulSoup(urlopen(url), "lxml")
    internalLinks = getInternalLinks(bs, url)
    externalLinks = getExternalLinks(bs, url)
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.append(link)
            print(link)
            
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.append(link)
            getAllExternalLinks(link)
            
allIntLinks.append('https://oreilly.com')
getAllExternalLinks('https://www.oreilly.com/')

https://play.google.com/store/apps/details?id=com.safariflow.queue
https://channelstore.roku.com/details/c8a2d0096693eb9455f6ac165003ee06/oreilly
https://oreilly.hk/
https://www.linkedin.com/company/oreilly-media
https://itunes.apple.com/us/app/safari-to-go/id881697395
https://oreillylearning.in/
https://learning.oreilly.com/start-trial/
https://learning.oreilly.com/search/?query=author%3A%22Ken%20Kousen%22&extended_publisher_data=true&highlight=true&include_assessments=false&include_case_studies=true&include_courses=true&include_playlists=true&include_collections=true&include_notebooks=true&include_sandboxes=true&include_scenarios=true&is_academic_institution_account=false&source=user&sort=date_added&facet_json=true&json_facets=true&page=0&include_facets=false
https://learning.oreilly.com/search/?query=author%3A%22Arianne%20Dee%22&extended_publisher_data=true&highlight=true&include_assessments=false&include_case_studies=true&include_courses=true&include_playlists=true&include_collecti

KeyboardInterrupt: 

In [12]:
len(allExtLinks), len(allIntLinks)

(129, 66)